In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as functional

import torchvision 
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display,clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import OrderedDict
from collections import namedtuple

In [2]:
train_set=torchvision.datasets.FashionMNIST(
    root="./data",train=True,download=True,
    transform=transforms.Compose([transforms.ToTensor()]))

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        
        self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=60)
        self.out=nn.Linear(in_features=60,out_features=10)
        
    def forward(self,t):
        # layer1-input layer
        t=t
        
        # layer2-hidden conv layer        
        t=self.conv1(t)
        t=functional.relu(t)
        t=functional.max_pool2d(t,kernel_size=2,stride=2)
        
        # layer3-hidden conv layer        
        t=self.conv2(t)
        t=functional.relu(t)
        t=functional.max_pool2d(t,kernel_size=2,stride=2)
        
        # layer4-hidden linear layer        
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=functional.relu(t)
        
        # layer5-hidden linear layer        
        t=self.fc2(t)
        t=functional.relu(t)
        
        # layer6-output layer        
        t=self.out(t)
        
        return t

In [4]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run=namedtuple("Run",params.keys())
        
        runs=[]
        for v in product(*params.values()):
            runs.append(Run(*v))
        
        return runs

In [5]:
class RunManager():
    def __init__(self):
        self.epoch_count=0
        self.epoch_loss=0
        self.epoch_num_correct=0
        self.epoch_start_time=None
        
        self.run_params=None
        self.run_count=0
        self.run_data=[]
        self.run_start_time=None
        
        self.network=None
        self.loader=None
        self.tb=None
        
    def begin_run(self,run,network,loader):
        self.run_start_time=time.time()
        self.run_params=run
        self.run_count+=1
        
        self.network=network
        self.loader=loader
        self.tb=SummaryWriter(comment=f"-{run}")
        
        images,labels=next(iter(self.loader))
        grid=torchvision.utils.make_grid(images)
        
        self.tb.add_image("images",grid)
        self.tb.add_graph(self.network,images)
        
    def end_run(self):
        self.tb.close()
        self.epoch_count=0
        
    def begin_epoch(self):
        self.epoch_start_time=time.time()
        self.epoch_count+=1
        self.epoch_loss=0
        self.epoch_num_correct=0
        
    def end_epoch(self):
        epoch_duration=time.time()-self.epoch_start_time
        run_duration=time.time()-self.run_start_time
        
        loss=self.epoch_loss/len(self.loader.dataset)
        accuracy=self.epoch_num_correct/len(self.loader.dataset)
        
        self.tb.add_scalar("Loss",loss,self.epoch_count)
        self.tb.add_scalar("Accuracy",accuracy,self.epoch_count)
        
        for name,param in self.network.named_parameters():
            self.tb.add_histogram(name,param,self.epoch_count)
            self.tb.add_histogram(f"{name}.grad",param.grad,self.epoch_count)
        
        results=OrderedDict()
        results["run"]=self.run_count
        results["epoch"]=self.epoch_count
        results["loss"]=loss
        results["accuracy"]=accuracy
        results["epoch duration"]=epoch_duration
        results["run duration"]=run_duration
        
        for k,v in self.run_params._asdict().items(): results[k]=v
        self.run_data.append(results)
        
        df=pd.DataFrame.from_dict(self.run_data,orient="columns")
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self,loss,batch):
        self.epoch_loss+=loss.item()*batch[0].shape[0]

    def track_num_correct(self,preds,labels):
        self.epoch_num_correct+=self._get_num_correct(preds,labels)

    @torch.no_grad()
    def _get_num_correct(self,preds,labels):
        return preds.argmax(dim=1).eq(labels).sum().item()

    def save(self,filename):
        pd.DataFrame.from_dict(self.run_data,orient="columns").to_csv(f"{filename}.csv")

        with open(f"{filename}.json","w",encoding="utf-8") as f:
            json.dump(self.run_data,f,ensure_ascii=False,indent=4)
            

In [6]:
params=OrderedDict(lr=[0.01],
                  batch_size=[1000,2000])

m=RunManager()
for run in RunBuilder.get_runs(params):
    network=Network()
    loader=DataLoader(train_set,batch_size=run.batch_size)
    optimizer=optim.Adam(network.parameters(),lr=run.lr)
    
    m.begin_run(run,network,loader)
    
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images,labels=batch
            preds=network(images)
            loss=functional.cross_entropy(preds,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss,batch)
            m.track_num_correct(preds,labels)
            
        m.end_epoch()
    m.end_run()
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size
0,1,1,1.012044,0.606350,9.432160,10.101192,0.01,1000
1,1,2,0.547645,0.790667,9.216863,19.426210,0.01,1000
2,1,3,0.453145,0.833950,9.160035,28.671629,0.01,1000
3,1,4,0.404460,0.853333,8.765798,37.532244,0.01,1000
4,1,5,0.376569,0.861800,9.225552,46.832293,0.01,1000
5,2,1,1.468307,0.427850,9.259208,10.549353,0.01,2000
6,2,2,0.706083,0.726933,8.891002,19.533964,0.01,2000
7,2,3,0.578521,0.772367,8.906311,28.523406,0.01,2000
8,2,4,0.503926,0.808967,8.339447,36.950089,0.01,2000
9,2,5,0.452868,0.833833,8.609289,45.645283,0.01,2000


In [7]:
params=OrderedDict(lr=[0.01],
                  batch_size=[1000,2000],
                  shuffle=[True,False])

m=RunManager()
for run in RunBuilder.get_runs(params):
    network=Network()
    loader=DataLoader(train_set,batch_size=run.batch_size,shuffle=run.shuffle)
    optimizer=optim.Adam(network.parameters(),lr=run.lr)
    
    m.begin_run(run,network,loader)
    
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images,labels=batch
            preds=network(images)
            loss=functional.cross_entropy(preds,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss,batch)
            m.track_num_correct(preds,labels)
            
        m.end_epoch()
    m.end_run()
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.034353,0.602050,9.729387,10.323858,0.01,1000,True
1,1,2,0.556507,0.782900,9.088155,19.518009,0.01,1000,True
2,1,3,0.462707,0.827983,9.052750,28.655520,0.01,1000,True
3,1,4,0.413005,0.847883,9.507171,38.237699,0.01,1000,True
4,1,5,0.365335,0.866383,9.396327,47.718816,0.01,1000,True
5,2,1,1.024408,0.606967,8.253441,8.875386,0.01,1000,False
6,2,2,0.567845,0.780717,8.644666,17.619482,0.01,1000,False
7,2,3,0.478429,0.819100,8.519245,26.218182,0.01,1000,False
8,2,4,0.425806,0.844383,9.373617,35.681458,0.01,1000,False
9,2,5,0.377567,0.862733,9.448394,45.212997,0.01,1000,False


#### DataLoader num_workers Test

In [8]:
params=OrderedDict(lr=[0.01],
                   batch_size=[100,1000,10000],
                   num_workers=[0,1,2,4,8,16]
                   #shuffle=[True,False]
                   )

m=RunManager()
for run in RunBuilder.get_runs(params):
    network=Network()
    loader=DataLoader(train_set,batch_size=run.batch_size,num_workers=run.num_workers)
    optimizer=optim.Adam(network.parameters(),lr=run.lr)
    
    m.begin_run(run,network,loader)
    
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            images,labels=batch
            preds=network(images)
            loss=functional.cross_entropy(preds,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss,batch)
            m.track_num_correct(preds,labels)
            
        m.end_epoch()
    m.end_run()
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers
0,1,1,0.610035,0.766317,10.937048,11.071450,0.01,100,0
1,2,1,0.538804,0.796300,13.345222,13.542481,0.01,100,1
2,3,1,0.572968,0.781717,13.789510,14.037174,0.01,100,2
3,4,1,0.587939,0.777200,13.648487,13.976110,0.01,100,4
4,5,1,0.561763,0.786917,13.400042,13.757281,0.01,100,8
5,6,1,0.584922,0.778033,13.539812,14.061024,0.01,100,16
6,7,1,1.218479,0.529767,9.235019,9.914929,0.01,1000,0
7,8,1,0.979347,0.625433,9.082731,9.840657,0.01,1000,1
8,9,1,0.933454,0.642833,8.048431,8.977778,0.01,1000,2
9,10,1,1.093146,0.581200,8.617688,9.437018,0.01,1000,4
